In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


In [4]:
lr = pd.read_csv("lr_results.csv")
rf = pd.read_csv("rf_results.csv")
xgb = pd.read_csv("xgb_results.csv")

print(lr.head())
print(rf.head())
print(xgb.head())

print(lr.shape)
print(rf.shape)
print(xgb.shape)

   drift   raw_acc   sig_acc   iso_acc  raw_conf  sig_conf  iso_conf  \
0    0.0  0.975808  0.999052  0.999386  0.946687  0.999151  0.999194   
1    0.2  0.961641  0.999052  0.999333  0.936695  0.999128  0.999058   
2    0.5  0.883484  0.999122  0.999368  0.919390  0.999009  0.997728   
3    1.0  0.759366  0.999140  0.998806  0.929867  0.998305  0.986430   

    raw_ent   sig_ent   iso_ent model  
0  0.155847  0.005721  0.003526    LR  
1  0.170174  0.005865  0.004291    LR  
2  0.192487  0.006614  0.009990    LR  
3  0.164378  0.010150  0.039404    LR  
   drift   raw_acc   sig_acc   iso_acc  raw_conf  sig_conf  iso_conf  \
0    0.0  0.999491  0.999508  0.999491  0.999401  0.999477  0.999355   
1    0.2  0.999421  0.999403  0.999403  0.999071  0.999453  0.999186   
2    0.5  0.999350  0.999368  0.999350  0.998303  0.999430  0.998724   
3    1.0  0.999122  0.999175  0.999140  0.995440  0.999365  0.996752   

    raw_ent   sig_ent   iso_ent  
0  0.002279  0.004112  0.003197  
1  0.00392

In [7]:
lr["model"] = "LR"
rf["model"] = "RF"
xgb["model"] = "XGB"

final = pd.concat([lr, rf, xgb], ignore_index=True)
final = final.sort_values(["model","drift"])
final.reset_index(drop=True, inplace=True)

final.head(12)


,drift,raw_acc,sig_acc,iso_acc,raw_conf,sig_conf,iso_conf,raw_ent,sig_ent,iso_ent,model
0,0.0,0.975808,0.999052,0.999386,0.946687,0.999151,0.999194,0.155847,0.005721,0.003526,LR
1,0.2,0.961641,0.999052,0.999333,0.936695,0.999128,0.999058,0.170174,0.005865,0.004291,LR
2,0.5,0.883484,0.999122,0.999368,0.919390,0.999009,0.997728,0.192487,0.006614,0.009990,LR
3,1.0,0.759366,0.999140,0.998806,0.929867,0.998305,0.986430,0.164378,0.010150,0.039404,LR
4,0.0,0.999491,0.999508,0.999491,0.999401,0.999477,0.999355,0.002279,0.004112,0.003197,RF
5,0.2,0.999421,0.999403,0.999403,0.999071,0.999453,0.999186,0.003925,0.004178,0.004088,RF
6,0.5,0.999350,0.999368,0.999350,0.998303,0.999430,0.998724,0.008087,0.004258,0.006525,RF
7,1.0,0.999122,0.999175,0.999140,0.995440,0.999365,0.996752,0.023106,0.004538,0.017075,RF
8,0.0,0.999579,0.999579,0.999561,0.999884,0.999483,0.999493,0.000486,0.003851,0.003075,XGB
9,0.2,0.999491,0.999544,0.999491,0.999832,0.999486,0.999382,0.000638,0.003880,0.003545,XGB


In [8]:
final.to_csv("final_results.csv", index=False)
print("Saved final_results.csv")


Saved final_results.csv
